In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch 

## Méthode de Gradient

In [4]:
def critere_1(f,e,x0,inter):
    x = tf.Variable(x0)
    
    norm = tf.norm(f(x))
    
    for k in range(inter):
        if norm < e:
            print(norm)
            return x, f(x)
 

In [262]:
def critere_2(f,e,x0,p,N):
    x = tf.Variable(x0)
    for k in range(inter):
        with tf.GradientTape(persistent=True) as tape:
            my_function = f(x,p)
        gradient = tape.gradient(my_function,x)
        del tape
        #gradient = tf.transpose(gradient)
        rx = tf.reduce_sum(gradient * x) / my_function
        if rx <= e:
            return x

    

In [263]:
#def critere_3(f,e,x0,inter):
    

In [466]:
def gradient_method(f,x0,p,N,e):
    
    alpha = 0.18
    
    Dk = np.eye(np.size(x0))
    Dk = tf.convert_to_tensor(Dk, dtype=tf.float64)
    
    x = tf.Variable(tf.convert_to_tensor(x0, dtype=tf.float64), dtype=tf.float64)
    x = tf.reshape(x,shape=(2,1))
    i = 0
    for k in range(N):
        i +=1
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            my_function = f(x,p)
        gradient = tape.gradient(my_function,x)
        
        gradient = tf.reshape(gradient, shape=(-1,1))
        
        x = x - alpha*(tf.linalg.matmul(Dk,gradient))
        critere_2(f,e,x,p,N)
        
    return x, i
    

In [467]:
def f(x,p):
    return (1-x[0])**2 + p*(x[1]-x[0]**2)**2


In [481]:
gradient_method(f,x0 = [0, 1.0],p=1, N = 50, e = 10^-3)


(<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
 array([[0.9911225 ],
        [0.97961258]])>,
 50)

## Méthode de Newton

In [245]:

# def newton_method(f,x0,N,e):
#     alpha = 1
    
#     x = tf.Variable(tf.convert_to_tensor(x0, dtype=tf.float64), dtype=tf.float64)
#     x = tf.reshape(x,shape=(2,1))
#     for i in range(N):
#         y = f(x)
#         with tf.GradientTape() as t2:
#             with tf.GradientTape() as t1:  
#                 dy_dx = t1.gradient(y, x)
#             print(dy_dx)
#             d2y_dx2 = t2.gradient(dy_dx, x)
        
#     return xk, f(xk)





In [446]:
def newton_method(f,x0,p,N,e):
    alpha = 1
    x = torch.tensor(x0, requires_grad=True, dtype=torch.float64)
    k = 0
    for k in range(N):
        k += 1
        y = f(x,p)

        grads = torch.autograd.grad(y, x, create_graph=True)
        
        H = torch.zeros((np.size(x0), np.size(x0)), dtype=torch.float64)
    
        for i in range(np.size(x0)):
            for j in range(np.size(x0)):
                H[i, j] = torch.autograd.grad(grads[0][i], x, retain_graph=True)[0][j]
                
        H_inv = torch.inverse(H)
        aux = x
        x = x - alpha * torch.matmul(H_inv, grads[0])
        result = torch.norm(x - aux,2).clone().detach()
        if result < e:
            return x.detach().numpy(),k
        
    return x.detach().numpy(),k

In [450]:
newton_method(f,x0 = [0, 1.0], p = 1, N = 100, e = 10**-3)


(array([1., 1.]), 8)

## Méthode cyclique des coordonnées

In [338]:
def newton_cyclique(X,p,vector, e, N, f):
    alpha = 0.1

    def derive1():
        return (f((X - (alpha + 0.0001) * vector) , p) - f((X - alpha * vector), p)) / 0.0001
    def derive2():
        return (f((X - (alpha + 0.0001) * vector) , p) + f((X - (alpha - 0.0001) * vector) , p) -2*f((X - alpha * vector), p)) / np.power(0.0001, 2)
    
    for k in range(N):
        dy_dx = derive1()
        d2y_dx2 = derive2()
        alphak = alpha - dy_dx / d2y_dx2

    
        if np.abs((alphak - alpha) / np.abs(alphak)) < e:
            return alphak

        alpha = alphak

    return alphak

In [475]:
def methode_cyclique(f,x0,p,e, N):
    i = 0
    n = 2
    e_canonique = [np.array([1, 0]), np.array([0, 1])]
    y = np.array(x0)
    j = 0

    for i in range(N):
        i += 1
        alpha = newton_cyclique(y,p, e_canonique[j], e, 20, f)
        yk = y - alpha * e_canonique[j]

        if (j == n-1):
            j = 0
        elif (j < n-1):
            j += 1

        if np.absolute(np.linalg.norm(yk - y)) < e:
            return yk, i

        y = yk

    return yk, i

methode_cyclique(f, x0 = [0, 1], p = 1, e = 1e-3, N = 50)

(array([1.00376207, 1.00928709]), 21)

In [455]:
%%time
p_values = [1, 20, 40, 60, 80, 100]
cyclique = []
newton = []
gradient = []
for p0 in p_values:
    print(p0)
    _,x1 = newton_method(f,x0 = [0, 1.0], p = p0, N = 50, e = 10^-3)
    _,x2 = methode_cyclique(f, x0 = [0, 1], p = p0, e = 1e-3, N = 50)
    _,x3 = gradient_method(f,x0 = [0, 1.0], p = p0, N = 50, e = 10^-3)
    
    gradient.append(x3)
    cyclique.append(x2)
    newton.append(x1)
    

1
20
40
60
80
100
CPU times: total: 35.1 s
Wall time: 35.2 s


In [456]:
print(cyclique)
print(newton)
print(gradient)

[21, 50, 50, 50, 50, 50]
[50, 50, 50, 50, 50, 50]
[50, 50, 50, 50, 50, 50]
